In [7]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import time
import datetime
import pandas as pd
import numpy as np
import requests

In [9]:
class GetBinanceData:
    @staticmethod
    def get_data(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        dataList = []
        current_date = datetime.datetime.now()
        start_date = startDate

        delta_units = GetBinanceData.convert_to_minutes(interval)
        print(delta_units)
        delta = datetime.timedelta(minutes=delta_units)

        while start_date < current_date:
            end_date = start_date + delta
            if end_date > current_date:
                end_date = current_date
            # Convert datetime objects to milliseconds
            start_ms = int(start_date.timestamp() * 1000)
            end_ms = int(end_date.timestamp() * 1000)

            start_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

            # print(f"Fetching data from {start_str} to {end_str}")
            data = GetBinanceData.get_historical_data(symbol, interval, start_ms, end_ms)
            if data:
                dataList.extend(data)
            start_date = end_date
            time.sleep(1)

        return dataList

    @staticmethod
    def get_historical_data(symbol, interval, start_time, end_time):
        url = f"https://api.binance.us/api/v3/klines"
        params = {
            'symbol': symbol.upper(),
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': 1000
        }
        # print(params)
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:

            print(f"Failed to fetch data. Status code: {response.json()}")
            return None

    @staticmethod
    def convert_to_minutes(time_str):
        time_mapping = {
            'm': 1,
            'h': 60,
            'd': 1440,
            'w': 10080,
            'M': 43200
        }

        num = int(time_str[:-1])
        unit = time_str[-1]

        return (num * time_mapping[unit])*1000

    def data_to_dataframe(data):
      # Define the column names based on Binance API response
      columns = [
          "date", "open", "high", "low", "close", "volume",
          "Close Time", "Quote Asset Volume", "Number of Trades",
          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
      ]

      # Convert the data to a pandas DataFrame
      df = pd.DataFrame(data, columns=columns)
      # # Convert timestamp columns to datetime
      # df["timestamp"] = pd.to_datetime(df['timestamp'])
      # df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms')
      # # Convert numeric columns to float
      numeric_columns = ["open", "high", "low", "close", "volume", "Quote Asset Volume",
                        "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
      df[numeric_columns] = df[numeric_columns].astype(float)
            # Drop the specified columns
      columns_to_drop = ["Close Time", "Quote Asset Volume", "Number of Trades",
                          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"]

      df.drop(columns=columns_to_drop, inplace=True)
      return df

    @staticmethod
    def get_data_dataFrame(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        data = GetBinanceData.get_data(symbol, interval, startDate, endDate)
        return GetBinanceData.data_to_dataframe(data)

In [10]:
timeperiod = '1m'

symbol = "btcusdt"
# marketData = GetBinanceData.get_data(symbol, timeperiod, startDate=datetime.datetime(2024, 6, 1), endDate=datetime.datetime.now())
marketData = GetBinanceData.get_data(symbol, timeperiod, startDate=datetime.datetime(2024, 4, 1), endDate=datetime.datetime(2024, 6, 1))


1000


In [12]:
data = GetBinanceData.data_to_dataframe(marketData)  # 1-minute data

In [13]:
fileName = "BTCUSDT_2024_04_01_to_21_06_2024.csv"

import pandas as pd



file_path = f'/content/drive/My Drive/{fileName}'
data.to_csv(file_path, index=False)
print(f"File saved to {file_path}")

File saved to /content/drive/My Drive/BTCUSDT_2024_04_01_to_21_06_2024.csv
